# Chương 7: Xây Dựng Ứng Dụng Chat
## Bắt Đầu Nhanh với Github Models API

Notebook này được điều chỉnh từ [Kho Mẫu Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), bao gồm các notebook truy cập dịch vụ [Azure OpenAI](notebook-azure-openai.ipynb).


# Tổng quan  
"Mô hình ngôn ngữ lớn là các hàm ánh xạ văn bản sang văn bản. Khi nhận một chuỗi văn bản đầu vào, mô hình ngôn ngữ lớn sẽ cố gắng dự đoán đoạn văn bản tiếp theo" (1). Notebook "bắt đầu nhanh" này sẽ giới thiệu cho người dùng các khái niệm LLM ở mức độ tổng quan, các yêu cầu gói cốt lõi để bắt đầu với AML, phần giới thiệu nhẹ về thiết kế prompt, cùng một số ví dụ ngắn về các trường hợp sử dụng khác nhau.


## Mục lục  

[Tổng quan](../../../../07-building-chat-applications/python)  
[Cách sử dụng Dịch vụ OpenAI](../../../../07-building-chat-applications/python)  
[1. Tạo Dịch vụ OpenAI của bạn](../../../../07-building-chat-applications/python)  
[2. Cài đặt](../../../../07-building-chat-applications/python)    
[3. Thông tin xác thực](../../../../07-building-chat-applications/python)  

[Các trường hợp sử dụng](../../../../07-building-chat-applications/python)    
[1. Tóm tắt văn bản](../../../../07-building-chat-applications/python)  
[2. Phân loại văn bản](../../../../07-building-chat-applications/python)  
[3. Tạo tên sản phẩm mới](../../../../07-building-chat-applications/python)  
[4. Tinh chỉnh bộ phân loại](../../../../07-building-chat-applications/python)  

[Tài liệu tham khảo](../../../../07-building-chat-applications/python)


### Tạo prompt đầu tiên của bạn  
Bài tập ngắn này sẽ giúp bạn làm quen với cách gửi prompt cho một mô hình trong Github Models với nhiệm vụ đơn giản là "tóm tắt".

**Các bước thực hiện**:  
1. Cài đặt thư viện `azure-ai-inference` vào môi trường python của bạn, nếu bạn chưa cài.  
2. Tải các thư viện hỗ trợ tiêu chuẩn và thiết lập thông tin xác thực cho Github Models.  
3. Chọn một mô hình phù hợp với nhiệm vụ của bạn  
4. Tạo một prompt đơn giản cho mô hình  
5. Gửi yêu cầu của bạn đến API của mô hình!


### 1. Cài đặt `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Tìm mô hình phù hợp  
Các mô hình GPT-3.5-turbo hoặc GPT-4 có khả năng hiểu và tạo ra ngôn ngữ tự nhiên.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Thiết kế Prompt  

"Điều kỳ diệu của các mô hình ngôn ngữ lớn là khi được huấn luyện để giảm thiểu lỗi dự đoán trên một lượng lớn văn bản, các mô hình này sẽ học được những khái niệm hữu ích cho việc dự đoán. Ví dụ, chúng học được các khái niệm như"(1):

* cách đánh vần
* cách sử dụng ngữ pháp
* cách diễn đạt lại câu
* cách trả lời câu hỏi
* cách trò chuyện
* cách viết bằng nhiều ngôn ngữ
* cách lập trình
* v.v.

#### Cách kiểm soát một mô hình ngôn ngữ lớn  
"Trong tất cả các đầu vào của một mô hình ngôn ngữ lớn, văn bản prompt là yếu tố có ảnh hưởng lớn nhất(1).

Các mô hình ngôn ngữ lớn có thể được hướng dẫn để tạo ra đầu ra theo một số cách:

Chỉ dẫn: Nói cho mô hình biết bạn muốn gì
Hoàn thành: Gợi ý để mô hình hoàn thành phần đầu của nội dung bạn muốn
Minh họa: Cho mô hình thấy bạn muốn gì, bằng cách:
Một vài ví dụ trong prompt
Hàng trăm hoặc hàng nghìn ví dụ trong bộ dữ liệu huấn luyện fine-tuning"



#### Có ba nguyên tắc cơ bản để tạo prompt:

**Chỉ rõ và minh họa**. Hãy làm rõ bạn muốn gì thông qua chỉ dẫn, ví dụ, hoặc kết hợp cả hai. Nếu bạn muốn mô hình sắp xếp một danh sách theo thứ tự chữ cái hoặc phân loại một đoạn văn theo cảm xúc, hãy cho nó thấy đó là điều bạn mong muốn.

**Cung cấp dữ liệu chất lượng**. Nếu bạn đang cố xây dựng một bộ phân loại hoặc muốn mô hình tuân theo một mẫu nhất định, hãy đảm bảo có đủ ví dụ. Đừng quên kiểm tra lại các ví dụ của bạn — mô hình thường đủ thông minh để nhận ra lỗi chính tả cơ bản và vẫn trả lời, nhưng nó cũng có thể cho rằng đó là chủ ý và điều này có thể ảnh hưởng đến kết quả.

**Kiểm tra các thiết lập.** Các thiết lập temperature và top_p kiểm soát mức độ xác định của mô hình khi tạo ra câu trả lời. Nếu bạn cần một câu trả lời mà chỉ có một đáp án đúng, hãy đặt các giá trị này thấp. Nếu bạn muốn có nhiều câu trả lời đa dạng hơn, có thể tăng các giá trị này lên. Sai lầm phổ biến nhất khi sử dụng các thiết lập này là nghĩ rằng chúng kiểm soát "độ thông minh" hoặc "sự sáng tạo" của mô hình.


Nguồn: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Tóm tắt văn bản  
#### Thử thách  
Tóm tắt văn bản bằng cách thêm 'tl;dr:' vào cuối một đoạn văn. Lưu ý cách mô hình hiểu và thực hiện nhiều nhiệm vụ mà không cần hướng dẫn bổ sung. Bạn có thể thử nghiệm với các prompt mô tả hơn thay cho tl;dr để điều chỉnh hành vi của mô hình và cá nhân hóa bản tóm tắt mà bạn nhận được(3).  

Những nghiên cứu gần đây đã cho thấy sự tiến bộ đáng kể trong nhiều nhiệm vụ và tiêu chuẩn NLP nhờ huấn luyện trước trên một tập dữ liệu văn bản lớn, sau đó tinh chỉnh cho từng nhiệm vụ cụ thể. Mặc dù kiến trúc thường không phụ thuộc vào nhiệm vụ, phương pháp này vẫn cần các bộ dữ liệu tinh chỉnh riêng biệt với hàng nghìn hoặc hàng chục nghìn ví dụ. Ngược lại, con người thường có thể thực hiện một nhiệm vụ ngôn ngữ mới chỉ với vài ví dụ hoặc chỉ dẫn đơn giản – điều mà các hệ thống NLP hiện tại vẫn còn gặp khó khăn. Ở đây, chúng tôi cho thấy rằng việc mở rộng quy mô các mô hình ngôn ngữ giúp cải thiện đáng kể hiệu suất với ít ví dụ, không phụ thuộc vào nhiệm vụ, thậm chí đôi khi còn đạt mức cạnh tranh với các phương pháp tinh chỉnh tiên tiến trước đó. 



Tl;dr


# Bài tập cho nhiều trường hợp sử dụng  
1. Tóm tắt văn bản  
2. Phân loại văn bản  
3. Tạo tên sản phẩm mới


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Phân loại Văn bản  
#### Thử thách  
Phân loại các mục vào các danh mục được cung cấp tại thời điểm suy luận. Trong ví dụ sau, chúng ta cung cấp cả các danh mục và văn bản cần phân loại trong prompt (*playground_reference).

Yêu cầu từ khách hàng: Xin chào, một trong các phím trên bàn phím laptop của tôi vừa bị hỏng gần đây và tôi cần thay thế:

Danh mục đã phân loại:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Tạo Tên Sản Phẩm Mới
#### Thử thách
Tạo tên sản phẩm từ các từ ví dụ. Ở đây, chúng tôi đưa vào phần gợi ý thông tin về sản phẩm mà chúng tôi sẽ tạo tên. Chúng tôi cũng cung cấp một ví dụ tương tự để thể hiện mẫu mà chúng tôi mong muốn nhận được. Ngoài ra, chúng tôi đã đặt giá trị temperature cao để tăng tính ngẫu nhiên và các phản hồi sáng tạo hơn.

Mô tả sản phẩm: Máy làm sữa lắc tại nhà
Từ khóa: nhanh, lành mạnh, nhỏ gọn.
Tên sản phẩm: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Mô tả sản phẩm: Một đôi giày có thể vừa với mọi cỡ chân.
Từ khóa: linh hoạt, vừa vặn, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Tài liệu tham khảo  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Ví dụ về OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Các phương pháp tốt nhất để tinh chỉnh GPT-3 phân loại văn bản](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Để Được Hỗ Trợ Thêm  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Người đóng góp
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm đối với bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
